In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, precision_recall_curve, auc


df = pd.read_csv('/content/creditcard.csv')
df = df.dropna()
scaler = StandardScaler()
features_scaled = scaler.fit_transform(df.drop('Class', axis=1))
features = features_scaled
labels = df['Class'].values
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


def geometric_mask(data, mask_rate=0.1):
    mask = np.random.geometric(p=mask_rate, size=data.shape)
    return np.where(mask > 1, 0, data)

x_train_augmented = geometric_mask(x_train)


def transformer_encoder(inputs):
    inputs = tf.expand_dims(inputs, -1)
    attention_output = layers.MultiHeadAttention(num_heads=4, key_dim=inputs.shape[-1], dropout=0.2)(inputs, inputs)
    attention_output = layers.Dropout(0.2)(attention_output)
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output + inputs)
    ffn_output = layers.Dense(units=inputs.shape[-1] * 2, activation='relu')(attention_output)
    ffn_output = layers.Dropout(0.2)(ffn_output)
    ffn_output = layers.Dense(units=inputs.shape[-1])(ffn_output)
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output + attention_output)
    return tf.squeeze(ffn_output, axis=-1)


input_layer = layers.Input(shape=(x_train.shape[1],))
encoded = transformer_encoder(input_layer)
decoded = layers.Dense(x_train.shape[1], activation='sigmoid')(encoded)
autoencoder = keras.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

class ContrastiveDiscriminator(keras.Model):
    def __init__(self):
        super(ContrastiveDiscriminator, self).__init__()
        self.discriminator_layers = keras.Sequential([
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])

    def call(self, inputs, training=False):
        return self.discriminator_layers(inputs, training=training)

    def contrastive_loss(self, real, fake, temperature=0.1):
        if len(real.shape) > 2:
            real = tf.reshape(real, [tf.shape(real)[0], -1])
        if len(fake.shape) > 2:
            fake = tf.reshape(fake, [tf.shape(fake)[0], -1])

        real_norm = tf.math.l2_normalize(real, axis=1)
        fake_norm = tf.math.l2_normalize(fake, axis=1)

        real_norm = tf.cast(real_norm, tf.float32)
        fake_norm = tf.cast(fake_norm, tf.float32)

        dot_product = tf.matmul(real_norm, fake_norm, transpose_b=True)
        exp_product = tf.exp(dot_product / temperature)

        diagonal = tf.linalg.diag_part(exp_product)
        loss = -tf.math.log(tf.reduce_sum(diagonal) / tf.reduce_sum(exp_product))
        return loss

class GAN(keras.Model):
    def __init__(self, autoencoder, discriminator):
        super(GAN, self).__init__()
        self.autoencoder = autoencoder
        self.discriminator = discriminator

    def compile(self, ae_opt, d_opt, loss_fn):
        super(GAN, self).compile()
        self.ae_opt = ae_opt
        self.d_opt = d_opt
        self.loss_fn = loss_fn

    def train_step(self, data):
        real_data = data

        with tf.GradientTape() as ae_tape, tf.GradientTape() as d_tape:
            reconstruction = self.autoencoder(real_data, training=True)

            ae_loss = self.loss_fn(real_data, reconstruction)

            real_output = self.discriminator(real_data, training=True)
            fake_output = self.discriminator(reconstruction, training=True)

            d_loss_real = self.loss_fn(tf.ones_like(real_output), real_output)
            d_loss_fake = self.loss_fn(tf.zeros_like(fake_output), fake_output)
            d_loss = 0.5 * (d_loss_real + d_loss_fake)

        ae_grads = ae_tape.gradient(ae_loss, self.autoencoder.trainable_variables)
        d_grads = d_tape.gradient(d_loss, self.discriminator.trainable_variables)

        self.ae_opt.apply_gradients(zip(ae_grads, self.autoencoder.trainable_variables))
        self.d_opt.apply_gradients(zip(d_grads, self.discriminator.trainable_variables))

        return {"ae_loss": ae_loss, "d_loss": d_loss}


discriminator = ContrastiveDiscriminator()
gan = GAN(autoencoder, discriminator)
gan.compile(
    ae_opt=keras.optimizers.Adam(learning_rate=0.001),
    d_opt=keras.optimizers.Adam(learning_rate=0.001),
    loss_fn=keras.losses.MeanSquaredError()
)
train_dataset = tf.data.Dataset.from_tensor_slices(x_train_augmented).batch(32)
gan.fit(train_dataset, epochs=50)


predictions = autoencoder.predict(x_test)
mse = np.mean(np.power(x_test - predictions, 2), axis=1)
threshold = np.percentile(mse, 90)  # Lowering the threshold to catch fewer, more likely anomalies
anomalies = mse > threshold

precision, recall, _ = precision_recall_curve(y_test, mse)
auc_score = auc(recall, precision)


print(f"Precision-Recall AUC: {auc_score}")
print(classification_report(y_test, anomalies))


Epoch 1/5
7121/7121 [==============================] - 104s 14ms/step - ae_loss: 0.1067 - d_loss: 9.5056e-04
Epoch 2/5
7121/7121 [==============================] - 98s 14ms/step - ae_loss: 0.1027 - d_loss: 2.0726e-04
Epoch 3/5
7121/7121 [==============================] - 93s 13ms/step - ae_loss: 0.1027 - d_loss: 0.0041
Epoch 4/5
7121/7121 [==============================] - 97s 14ms/step - ae_loss: 0.1027 - d_loss: 0.0020
Epoch 5/5
1781/1781 [==============================] - 11s 6ms/step
Precision-Recall AUC: 0.1625730737170408
              precision    recall  f1-score   support

           0       1.00      0.90      0.95     56864
           1       0.02      0.92      0.03        98

    accuracy                           0.90     56962
   macro avg       0.51      0.91      0.49     56962
weighted avg       1.00      0.90      0.95     56962

